In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
import os
from typing import Union, Tuple

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

In [3]:
df = pd.read_csv("data/EMEWS_cleaned_with_nan.csv")

In [4]:
df.head()

,day,date,total_number_of_patients,total_number_of_emews,zone_a_mwr_patients,zone_a_mwr_cat_3,zone_a_mwr_cat_4,zone_a_mwr_sets_of_emews,zone_a_mwr_deescalations,zone_a_mwr_escalations,...,zone_a__cat_3,zone_a__sets_of_emews,zone_a__deescalations,zone_a__escalations,zone_b/c_patients,zone_b/c_cat_2,zone_b/c_cat_3,zone_b/c_sets_of_emews,zone_b/c_deescalations,zone_b/c_escalations
0,thursday,2024-01-04 00:00:00,100.0,302.0,64.0,53.0,11.0,192.0,10.0,6.0,...,9.0,50.0,9.0,1.0,15.0,NaN,NaN,60.0,1.0,5.0
1,friday,2024-01-05 00:00:00,112.0,220.0,59.0,59.0,0.0,90.0,10.0,0.0,...,12.0,60.0,10.0,0.0,27.0,12.0,15.0,70.0,0.0,5.0
2,saturday,2024-01-06 00:00:00,69.0,175.0,36.0,36.0,0.0,58.0,3.0,2.0,...,0.0,67.0,5.0,2.0,15.0,NaN,NaN,50.0,0.0,5.0
3,sunday,2024-01-07 00:00:00,74.0,206.0,30.0,27.0,3.0,41.0,2.0,11.0,...,6.0,61.0,7.0,3.0,16.0,NaN,NaN,50.0,0.0,5.0
4,monday,2024-01-08 00:00:00,87.0,267.0,41.0,38.0,3.0,116.0,10.0,1.0,...,5.0,73.0,8.0,1.0,23.0,10.0,13.0,78.0,0.0,4.0


In [5]:
# X = df.drop(columns=['total_number_of_patients'])  # Drop date unless you plan to extract features
# y = df[['total_number_of_patients']]

X = df.drop(columns=['total_number_of_patients', 'total_number_of_emews'])
y = df[['total_number_of_patients', 'total_number_of_emews']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [9]:
def apply_imputation(
        X_train: pd.DataFrame, 
        X_test: pd.DataFrame,
        imputer: Union[KNNImputer, IterativeImputer, SimpleImputer]
        ) -> Tuple[pd.DataFrame, pd.DataFrame]:

    num_cols = X_train.select_dtypes(include='number').columns
    non_num_cols = X_train.columns.difference(num_cols).to_list()

    X_train_num = X_train[num_cols]
    X_test_num = X_test[num_cols]

    needs_scaling = type(imputer) in (KNNImputer, IterativeImputer)

    if needs_scaling:
        scaler = StandardScaler()
        X_train_num = scaler.fit_transform(X_train_num)
        X_test_num = scaler.transform(X_test_num)

    X_train_imputed = imputer.fit_transform(X_train_num)
    X_test_imputed = imputer.transform(X_test_num)

    if needs_scaling:
        X_train_imputed = scaler.inverse_transform(X_train_imputed)
        X_test_imputed = scaler.inverse_transform(X_test_imputed)

    X_train_imputed = np.round(X_train_imputed).astype(int)
    X_test_imputed = np.round(X_test_imputed).astype(int)

    # Replace only numeric columns
    X_train_copy = X_train.copy()
    X_train_copy[num_cols] = X_train_imputed
    X_train_copy[non_num_cols] = X_train[non_num_cols]

    X_test_copy = X_test.copy()
    X_test_copy[num_cols] = X_test_imputed
    X_test_copy[non_num_cols] = X_test[non_num_cols]

    return X_train_copy, X_test_copy

In [10]:
imputation_strategies = {
    'mean': SimpleImputer(strategy='mean'),
    'median': SimpleImputer(strategy='median'),
    'mode': SimpleImputer(strategy='most_frequent'),
    'mice': IterativeImputer(random_state=42),
    'mice_hgb': IterativeImputer(estimator=HistGradientBoostingRegressor(random_state=42), random_state=42),
    'mice_lr': IterativeImputer(estimator=LinearRegression(), random_state=42),
    'mice_rf': IterativeImputer(estimator=RandomForestRegressor(random_state=42), random_state=42),
    'mice_svr': IterativeImputer(estimator=SVR(), random_state=42)
}

for i in range(3, 6):
    key = f'knn_{i}'
    imputation_strategies[key] = KNNImputer(n_neighbors=i)
    key = f'knn_{i}_distance'
    imputation_strategies[key] = KNNImputer(n_neighbors=i, weights='distance')
    key = f'mice_knn_{i}'
    imputation_strategies[key] = IterativeImputer(estimator=KNeighborsRegressor(n_neighbors=i), random_state=42)
    key = f'mice_knn_{i}_distance'
    imputation_strategies[key] = IterativeImputer(estimator=KNeighborsRegressor(n_neighbors=i, weights='distance'), random_state=42)
    

datasets = {}

for name, imputer in imputation_strategies.items():
    X_train_imp, X_test_imp = apply_imputation(X_train, X_test, imputer)

    train_df = pd.concat([X_train_imp, y_train], axis=1)
    test_df = pd.concat([X_test_imp, y_test], axis=1)

    train_df['date'] = pd.to_datetime(train_df['date'])
    test_df['date'] = pd.to_datetime(test_df['date'])

    # manual checks...
    patient_count_cols = ['zone_a_mwr_patients', 'zone_a__patients', 'zone_b/c_patients']
    emews_count_cols = ['zone_a_mwr_sets_of_emews', 'zone_a__sets_of_emews', 'zone_b/c_sets_of_emews']
    
    train_df['total_number_of_patients'] = train_df['total_number_of_patients'].fillna(train_df[patient_count_cols].sum(axis=1))
    test_df['total_number_of_patients'] = test_df['total_number_of_patients'].fillna(test_df[patient_count_cols].sum(axis=1))

    train_df['total_number_of_emews'] = train_df['total_number_of_emews'].fillna(train_df[emews_count_cols].sum(axis=1))
    test_df['total_number_of_emews'] = test_df['total_number_of_emews'].fillna(test_df[emews_count_cols].sum(axis=1))

    # train_df['corrected_total_number_of_patients'] = train_df[patient_count_cols].sum(axis=1)
    # test_df['corrected_total_number_of_patients'] = test_df[patient_count_cols].sum(axis=1)

    datasets[name] = (train_df, test_df)

In [ ]:
from pycaret.regression import setup, compare_models, pull
import pandas as pd

results = []

for name, (train_df, test_df) in datasets.items():
    print(f"\n🔍 Processing dataset: {name}")
    # train_df = train_df.copy().drop(columns=['corrected_total_number_of_patients'])
    # test_df = test_df.copy().drop(columns=['corrected_total_number_of_patients'])

    setup(
        data=train_df,
        target='total_number_of_patients',
        session_id=42,
        verbose=False,
        test_data=test_df  # Use predefined test set
    )

    best_model = compare_models(exclude=['lightgbm'], sort='R2')
    leaderboard = pull()
    top_model_results = leaderboard.iloc[0]

    results.append({
        'dataset': name,
        'model': top_model_results['Model'],
        'R2': top_model_results['R2'],
        'MAE': top_model_results['MAE'],
        'RMSE': top_model_results['RMSE']
    })

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='R2', ascending=False)
print("\n📊 Final Results:")
print(results_df)



🔍 Processing dataset: mean


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,3.6747,36.7457,5.6111,0.9369,0.2600,0.1170,0.0170
en,Elastic Net,3.6482,36.7608,5.6157,0.9369,0.2599,0.1163,0.0170
llar,Lasso Least Angle Regression,3.6749,36.7463,5.6112,0.9369,0.2600,0.1170,0.0160
br,Bayesian Ridge,3.5909,37.3292,5.6662,0.9358,0.2623,0.1151,0.0160
lr,Linear Regression,3.5864,37.8851,5.7190,0.9350,0.2633,0.1134,0.0190
ridge,Ridge Regression,3.5819,37.8567,5.7153,0.9350,0.2630,0.1132,0.0160
lar,Least Angle Regression,3.6133,38.0127,5.7151,0.9346,0.2647,0.1141,0.0170
catboost,CatBoost Regressor,3.3589,44.6625,6.1910,0.9239,0.2503,0.0752,0.5950
huber,Huber Regressor,2.8549,45.7809,6.1727,0.9213,0.2855,0.0686,0.0210
gbr,Gradient Boosting Regressor,3.8967,46.2853,6.4184,0.9199,0.2700,0.0980,0.0330



🔍 Processing dataset: median


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
en,Elastic Net,3.6604,37.4212,5.6429,0.9358,0.2649,0.1161,0.0190
br,Bayesian Ridge,3.5942,37.6438,5.6376,0.9355,0.2649,0.1139,0.0190
lasso,Lasso Regression,3.7173,37.6933,5.6825,0.9353,0.2659,0.1177,0.0150
llar,Lasso Least Angle Regression,3.7171,37.6927,5.6824,0.9353,0.2659,0.1177,0.0180
ridge,Ridge Regression,3.5993,38.3510,5.7127,0.9343,0.2758,0.1142,0.0170
lr,Linear Regression,3.5995,38.4017,5.7176,0.9342,0.2751,0.1140,0.0190
lar,Least Angle Regression,3.7429,43.2117,6.1357,0.9255,0.2778,0.1160,0.0180
catboost,CatBoost Regressor,3.4235,45.8061,6.2615,0.9224,0.2558,0.0767,0.5330
huber,Huber Regressor,2.8247,48.3825,6.3592,0.9168,0.3022,0.0673,0.0200
gbr,Gradient Boosting Regressor,3.8877,48.4347,6.5618,0.9160,0.2567,0.0952,0.0400



🔍 Processing dataset: mode


...

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,4.3740,42.6583,6.2626,0.9259,0.2312,0.1362,0.0180
llar,Lasso Least Angle Regression,4.3741,42.6584,6.2626,0.9259,0.2312,0.1362,0.0150
en,Elastic Net,4.3877,42.8401,6.2745,0.9256,0.2296,0.1368,0.0160
br,Bayesian Ridge,4.4542,44.0167,6.3644,0.9235,0.2283,0.1394,0.0160
lr,Linear Regression,4.4759,44.4863,6.3955,0.9230,0.2377,0.1401,0.0170
ridge,Ridge Regression,4.4721,44.5076,6.3946,0.9229,0.2341,0.1398,0.0180
lar,Least Angle Regression,4.5060,44.9719,6.4243,0.9221,0.2364,0.1411,0.1370
huber,Huber Regressor,4.2242,48.2723,6.6356,0.9165,0.2501,0.1288,0.0190
catboost,CatBoost Regressor,4.3909,56.0099,7.0595,0.9038,0.2745,0.1121,0.4970
gbr,Gradient Boosting Regressor,4.6177,55.6503,7.1103,0.9028,0.2853,0.1209,0.0330



🔍 Processing dataset: mice_knn_3_distance


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,05:26:53
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,4.2572,40.8190,6.1028,0.9290,0.2593,0.1290,0.0220
llar,Lasso Least Angle Regression,4.2572,40.8188,6.1028,0.9290,0.2593,0.1290,0.0360
en,Elastic Net,4.2622,41.0959,6.1248,0.9285,0.2601,0.1295,0.0220
br,Bayesian Ridge,4.2813,41.4527,6.1510,0.9278,0.2619,0.1308,0.0240
lr,Linear Regression,4.2800,41.9137,6.1748,0.9271,0.2690,0.1310,0.0220
ridge,Ridge Regression,4.2755,41.8994,6.1725,0.9271,0.2688,0.1308,0.0280
lar,Least Angle Regression,4.3445,43.4800,6.2863,0.9243,0.2906,0.1338,0.0230
huber,Huber Regressor,4.0653,45.3107,6.3904,0.9211,0.2393,0.1239,0.0290
knn,K Neighbors Regressor,5.8924,76.7392,8.5481,0.8680,0.3301,0.1503,0.0290
omp,Orthogonal Matching Pursuit,7.4851,104.6684,10.0864,0.8210,0.3587,0.2306,0.0390


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

In [110]:
from pycaret.regression import setup, compare_models, pull
import pandas as pd

results = []

for name, (train_df, test_df) in datasets.items():
    print(f"\n🔍 Processing dataset: {name}")
    # train_df = train_df.copy().drop(columns=['corrected_total_number_of_patients'])
    # test_df = test_df.copy().drop(columns=['corrected_total_number_of_patients'])

    setup(
        data=train_df,
        target='total_number_of_emews',
        session_id=42,
        verbose=False,
        test_data=test_df  # Use predefined test set
    )

    best_model = compare_models(exclude=['lightgbm'], sort='R2')
    leaderboard = pull()
    top_model_results = leaderboard.iloc[0]

    results.append({
        'dataset': name,
        'model': top_model_results['Model'],
        'R2': top_model_results['R2'],
        'MAE': top_model_results['MAE'],
        'RMSE': top_model_results['RMSE']
    })

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='R2', ascending=False)
print("\n📊 Final Results:")
print(results_df)



🔍 Processing dataset: mean


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,8.6691,304.1862,15.9037,0.9491,0.3259,0.0705,0.0230
llar,Lasso Least Angle Regression,8.6697,304.1956,15.9041,0.9491,0.3259,0.0705,0.0180
en,Elastic Net,8.6992,304.5382,15.9383,0.9490,0.3311,0.0706,0.0220
br,Bayesian Ridge,8.7277,307.7215,16.0622,0.9484,0.3376,0.0715,0.0200
ridge,Ridge Regression,8.8944,326.6865,16.6220,0.9450,0.3277,0.0721,0.0200
lr,Linear Regression,8.9023,327.6604,16.6462,0.9449,0.3276,0.0722,0.0200
huber,Huber Regressor,7.3708,343.3379,17.0881,0.9428,0.3363,0.0544,0.0220
gbr,Gradient Boosting Regressor,11.7813,391.8530,19.1050,0.9342,0.2795,0.0896,0.0320
lar,Least Angle Regression,9.3326,403.6277,18.2039,0.9313,0.3416,0.0741,0.0230
et,Extra Trees Regressor,11.8334,480.5326,21.2279,0.9207,0.2048,0.0779,0.0520



🔍 Processing dataset: median


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,8.2569,284.2602,15.3108,0.9525,0.3318,0.0679,0.0170
llar,Lasso Least Angle Regression,8.2576,284.2633,15.3113,0.9525,0.3318,0.0679,0.0210
en,Elastic Net,8.2737,284.4475,15.3422,0.9524,0.3327,0.0678,0.0230
br,Bayesian Ridge,8.2932,284.3211,15.3654,0.9524,0.3321,0.0684,0.0180
ridge,Ridge Regression,8.5953,292.9245,15.7395,0.9510,0.3364,0.0730,0.0170
lr,Linear Regression,8.6027,293.2936,15.7527,0.9509,0.3370,0.0731,0.0220
huber,Huber Regressor,7.1509,336.8244,16.8964,0.9438,0.3635,0.0539,0.0200
gbr,Gradient Boosting Regressor,11.3545,366.5766,18.4312,0.9383,0.2684,0.0881,0.0450
lar,Least Angle Regression,9.5789,406.3509,18.4382,0.9304,0.3474,0.0848,0.0180
et,Extra Trees Regressor,11.3945,453.6491,20.6511,0.9241,0.1988,0.0761,0.0480



🔍 Processing dataset: mode


...


🔍 Processing dataset: mice_knn_5


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,9.9828,295.3481,16.2521,0.9497,0.2449,0.0898,0.0180
llar,Lasso Least Angle Regression,9.9827,295.3530,16.2522,0.9497,0.2449,0.0898,0.0180
en,Elastic Net,10.0229,297.5547,16.3190,0.9493,0.2451,0.0900,0.0170
br,Bayesian Ridge,10.1531,301.6415,16.4536,0.9486,0.2577,0.0916,0.0180
lr,Linear Regression,10.3983,306.0890,16.6095,0.9478,0.2823,0.0944,0.0180
ridge,Ridge Regression,10.3898,305.9409,16.6037,0.9478,0.2820,0.0943,0.0160
huber,Huber Regressor,9.7049,321.1965,17.1538,0.9458,0.2502,0.0864,0.0220
lar,Least Angle Regression,10.6238,335.9603,17.3649,0.9426,0.3020,0.0993,0.0160
gbr,Gradient Boosting Regressor,13.0347,481.7000,21.3054,0.9174,0.3216,0.0990,0.0340
et,Extra Trees Regressor,12.8015,542.6514,22.5793,0.9093,0.2199,0.0895,0.0500



🔍 Processing dataset: mice_knn_5_distance


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,10.8766,354.8450,17.5486,0.9399,0.4071,0.0934,0.0180
llar,Lasso Least Angle Regression,10.8765,354.8466,17.5486,0.9399,0.4071,0.0934,0.0170
en,Elastic Net,10.9080,355.3647,17.5744,0.9398,0.4068,0.0934,0.0170
br,Bayesian Ridge,10.9528,355.9885,17.6133,0.9397,0.4034,0.0945,0.0220
lar,Least Angle Regression,11.3249,367.1260,17.9924,0.9379,0.4096,0.1010,0.0160
ridge,Ridge Regression,11.3567,369.1256,18.0591,0.9375,0.4044,0.1008,0.0170
lr,Linear Regression,11.3858,370.1038,18.0933,0.9373,0.4051,0.1010,0.0200
huber,Huber Regressor,10.5167,373.6304,18.0769,0.9369,0.4015,0.0874,0.0210
gbr,Gradient Boosting Regressor,13.5608,529.1639,22.2346,0.9095,0.3151,0.1015,0.0340
et,Extra Trees Regressor,13.5471,608.3969,23.8086,0.8982,0.2229,0.0935,0.0500



📊 Final Results:
                dataset             model      R2      MAE     RMSE
3                  mice  Lasso Regression  0.9562   8.0355  14.4622
2                  mode       Elastic Net  0.9532   9.6378  15.8541
1                median  Lasso Regression  0.9525   8.2569  15.3108
5               mice_lr    Bayesian Ridge  0.9511   8.6472  15.5656
18           mice_knn_5  Lasso Regression  0.9497   9.9828  16.2521
0                  mean  Lasso Regression  0.9491   8.6691  15.9037
7              mice_svr  Lasso Regression  0.9460  10.6197  16.8736
14           mice_knn_4  Lasso Regression  0.9445  10.7207  17.1908
16                knn_5  Lasso Regression  0.9440  11.2473  17.2263
17       knn_5_distance  Lasso Regression  0.9432  11.2882  17.3629
11  mice_knn_3_distance  Lasso Regression  0.9430  10.8612  17.2654
15  mice_knn_4_distance  Lasso Regression  0.9423  11.0857  17.4178
10           mice_knn_3  Lasso Regression  0.9417  11.3093  17.6228
12                knn_4  Lasso

In [ ]:
# for name, (train_df, test_df) in datasets.items():
#     path = f'data/imputed/'
#     os.makedirs(path, exist_ok=True)

#     df = pd.concat([train_df, test_df], axis=0).sort_values(by='date').reset_index(drop=True)

#     df.to_csv(path+f'{name}_df.csv', index=False)